### Because the way to scrape the information about another section is as same as these codes, we only show the codes of scraping the information about course reserve materials in communication science section.

In [ ]:
from selenium import webdriver  
import time 
from lxml import etree
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
import pandas as pd
import requests
import bs4 as bs

LIBRARY_HOME_PAGE_URL = "https://julac.hosted.exlibrisgroup.com/primo-explore/search?query=sub,contains,social,OR&query=sub,contains,sociology,AND&tab=default_tab&search_scope=Course&sortby=rank&vid=CUHK&lang=en_US&mode=advanced&offset=0" 
PATIENCE_TIME = 60 
LOAD_MORE_BUTTON_XPATH = '//*[@id="mainResults"]/div[2]/div/div[11]/button' 
source = requests.get(LIBRARY_HOME_PAGE_URL)

driver = webdriver.Chrome() 
driver.get(LIBRARY_HOME_PAGE_URL) 
i=0
while True: 
    try: 
        loadMoreButton = driver.find_element_by_xpath("//button[contains(.,'Load more results')]") 
        time.sleep(2) 
        loadMoreButton.click() 
        print("success!")
        time.sleep(6)
        i = i+1
        if i==54:
            break
    except Exception as e: 
        print (e) 
        break 
print ("Complete")
time.sleep(10) 

print('开始抓取......\n')
titles = []
authors = []
publishers = []
years = []
x = 1
while x <= 532:
    books_containers = driv er.find_elements_by_xpath('//*[@id="mainResults"]/div[2]/div/div[' + str(x) + ']')
    for container in books_containers:
        x += 1
        try:
            # 抓取书名 + 作者
            title_author = container.find_elements_by_tag_name("h3")[0].text
            pair = title_author.split('\n/')
            # 没有作者
            if len(pair) == 1:
                title = pair[0]
                author = ''
            elif len(pair) == 2:
                title, author = pair
            else:
                title = pair[0]
                author = ', '.join(pair[1:])
            titles.append(title)
            authors.append(author)
            print('title:', title)
            print('author:', author)

            # 抓取出版社，可能有多个出版社
            pubs = [pub.text for pub in container.find_elements_by_class_name("item-detail")][-1]
            print('pubs', pubs)
            publishers.append(pubs)

            # c2018
            year = pubs.split(' ')[-1].replace('c', '').replace('-', '')
            year = year if '1000' < year <= '2018' else ''
            print('year:', year)
            years.append(year)
            print('一本....\n')
        except Exception as e:
            print('这里报错了。。，需要调试下... 异常如下：{}\n'.format(str(e)))

print("done")

driver.quit()



In [16]:
dic = {
    'titles': titles,
    'publishers': publishers,
    'authors': authors,
    'years': years,
}
   
pd_form = pd.DataFrame.from_dict(dic, orient = 'index').transpose()

pd_form.to_csv('cuhk media.csv')
#pd_form